<a href="https://colab.research.google.com/github/tsakailab/cisexpkit/blob/master/Experiment/colab/pc_plane_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ポイントクラウドを用いた平面の検出


Q21: ベクトル $\bf a$ のノルム（長さ）を $\|\bf a\|$と記す．また，ベクトル $\bf a$ と $\bf b$の内積を ${\bf a}\cdot{\bf b}$，外積を ${\bf a}\times{\bf b}$ と記す．__これらの記法を用いて__，「位置ベクトル ${\bf p}_0$，${\bf p}_1$，${\bf p}_2$ の3点を通る平面」と「位置ベクトル $\bf p$ の点」の間の距離 $d$ を求める公式を作れ．

Q22: Q21の公式を図を用いて解説せよ．

### [ポイントクラウドが保存されたファイル](https://github.com/tsakailab/cisexpkit/raw/master/Experiment/colab/pc_octahedron.zip)をダウンロードします．
能動的ステレオカメラRealsense SR300で取得したカラー画像と深度画像および逆透視変換したポイントクラウドのファイルが含まれています．

In [ ]:
import zipfile
import os
zipURL = "https://github.com/tsakailab/cisexpkit/raw/master/Experiment/colab/pc_octahedron.zip"
!wget $zipURL --no-check-certificate --show-progress -q -O "/tmp/pc_octahedron.zip"
with zipfile.ZipFile("/tmp/pc_octahedron.zip", 'r') as f:
    f.extractall("/tmp")
!ls /tmp/

In [ ]:
import numpy as np
from PIL import Image
img = np.asarray(Image.open("/tmp/color00pc.png"))

%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(img)

## Open3Dでply形式のポイントクラウドを読み込み，plotlyで表示します．

In [ ]:
!pip install -q open3d
import open3d as o3d

In [ ]:
pcd = o3d.io.read_point_cloud("/tmp/xyzrgb0"+str(int(np.random.choice(6,1)[0]))+"pc.ply")
points = np.asarray(pcd.points)

nd = points.shape[0]
n = 30000
p = np.random.choice(nd, min(n,nd), replace=False)
print("%d out of %d points are displayed." % (n, nd))

import plotly.graph_objs  as go
xyz = points[p,:]
rgb = np.asarray(pcd.colors)[p,:] * 1.5 # brighter

trace = go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers',
                     marker=dict(size=2, 
                                color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(rgb[:,0], rgb[:,1], rgb[:,2])],
                                opacity=0.5))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)

fig.show()

## 最も多い点で表された平面の法線と通る点を推定する関数`DetectPlane`を定義します．
[[Tarsha-Kurdi+08]](https://halshs.archives-ouvertes.fr/halshs-00278397/document)によるRANSACアルゴリズムを参考に実装したものです．

入力:
> `points`:  n行3列のNumPy配列．nは点の数，各列はX,Y,Z座標を表します．

> `n_trials`:  試行回数（規定値30回）

> `th`:  面からの距離の閾値（規定値3mm）

出力
> `plane`:  推定した平面のパラメタ．

>> `plane["normal"]`:  法線ベクトル

>> `plane["p3idx"]`:  通る3点の番号．面は `points[plane["p3idx"][0]]`，`points[plane["p3idx"][1]]`，`points[plane["p3idx"][2]]`の3点を通ります．

Q23: [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus)とは何か．原理と特長を述べよ．

Q24: 関数`DetectPlane`が平面の法線と通る点を推定する仕組みを解説せよ．


In [ ]:
def DetectPlane(points, n_trials=30, th=3):

    # initial settings
    plane = dict(normal=None, p3idx=None)
    n_max, dev_min = 0, float("inf")

    for i in range(n_trials):
        # randomly pick up three points
        p3idx = np.random.choice(points.shape[0], 3, replace=False)

        # compute a unit normal vector
        normal = np.cross(points[p3idx[1]] - points[p3idx[0]], points[p3idx[2]] - points[p3idx[0]])
        normal = normal / np.linalg.norm(normal)

        # compute distances from the plane with a point p3idx[0] and the normal vector
        distances = np.abs(np.dot(points - points[p3idx[0],:], normal))

        # find the neighboring points to the plane
        pidx_neighbors = np.where(distances < th)[0]
        num_neighbors = len(pidx_neighbors)
        deviation = np.std(distances[pidx_neighbors])

        # check if the plane is better than the current estimate
        if num_neighbors > n_max or (num_neighbors == n_max and deviation < dev_min):
            n_max, dev_min = num_neighbors, deviation
            plane["normal"], plane["p3idx"] = normal, p3idx

    return plane

### ポイントクラウドに適用して，最大の平面を検出します．

Q25: 表示されるヒストグラムは何を表しているか．このヒストグラムから何がわかるか．

In [ ]:
plane1 = DetectPlane(points, n_trials=100, th=5)

print("Estimated unit normal vector =", plane1["normal"])
distances1 = np.abs(np.dot(points - points[plane1["p3idx"][0],:], plane1["normal"]))
import matplotlib.pyplot as plt
_ = plt.hist(distances1, bins=50)
plt.xlabel("Distance [mm]")

### 検出した平面に近い点を着色して表示します．

Q26: 「検出」の誤りは2種類ある．誤検出（false positive detection）と検出漏れ（false negative detection）とは何か．平面の検出における誤検出と検出漏れの例を具体的に示し，それらの原因と根拠を述べよ．

Q27: 入力の `n_trials` や `th` の値が大きい・小さいと，`DetectPlane`による平面の検出結果はどうなるか．表示される図を用いて説明せよ．また，その結果になる原因を考察せよ．
> 前のセル（`plane1 = ...`）で入力を変えて実行し，次のセル（`# plot the neighboring ...`）で表示・観察する．

> 入力が同じでも`DetectPlane`は実行する毎に異なる結果を出力することがある．反復して観察すること． 

In [ ]:
# plot the neighboring points to the plane within 10mm
disp_mm = 10.
pidx_on_plane1 = np.where(distances1 < disp_mm)[0]
p1 = np.intersect1d(p, pidx_on_plane1)
print("Points within %2.0f mm of the plane are shown in green." % (disp_mm))
xyz = points[p1,:]

trace_p1 = go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers',
                           marker=dict(size=2, color='rgb(0,255,0)',
                           opacity=0.05))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace, trace_p1], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)

fig.show()

### 検出した平面に近い点を除いたポイントクラウドから，再び平面を検出します．

In [ ]:
points1 = np.delete(points, pidx_on_plane1, axis=0)

plane2 = DetectPlane(points1, n_trials=100, th=5)

print("Estimated unit normal vector =", plane2["normal"])
distances2 = np.abs(np.dot(points - points[plane2["p3idx"][0],:], plane2["normal"]))
import matplotlib.pyplot as plt
_ = plt.hist(distances2, bins=50)
plt.xlabel("Distance [mm]")

### 検出した平面に近い点を着色して表示します．

Q28: 検出される2つ目の平面について報告せよ．前のセル（`points1 = ...`）と次のセル（`# plot the neighboring ...`）の実行を何度か繰り返すこと．

In [ ]:
# plot the neighboring points to the plane within 10mm
disp_mm = 10.
pidx_on_plane2 = np.where(distances2 < disp_mm)[0]
p2 = np.intersect1d(p, pidx_on_plane2)
print("Points within %2.0f mm of the 2nd plane are shown in red." % (disp_mm))
xyz = points[p2,:]

trace_p2 = go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers',
                           marker=dict(size=2, color='rgb(255,0,0)',
                           opacity=0.05))

layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=[trace, trace_p1, trace_p2], layout=layout)
camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=-0.4, z=0), eye=dict(x=0, y=0.8, z=-2))
fig.update_layout(scene_camera=camera)

fig.show()

# 最終版のレポートを作成しましょう．

1日目と2日目のレポートと，この3日目の内容をあわせた最終版のレポートを作成し，提出してください．
* 1日目・2日目のレポートには，積極的に加筆・修正等の改善を施してください．
* レポートの書き方を身に着けるため，[実験レポートの例](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/Document/report_sample_cover.pdf)を大いに参考にしてください．
* 提出前に，[チェックシート](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/Document/report_checksheet_n.pdf)で必ず確認してください．

Q29: 提出済みの1日目・2日目のレポートにどのような改善を施したか．主な変更点を述べよ．

Q30: 実験中に気づいたこと，工夫したこと，実験を通して得られたこと，今後学んでみたいこと等，感想や意見を書いてください．また，他の学生と協働したことがあれば報告してください．